In [1]:
import os
import trimesh
import random

from tqdm import tqdm

from scripts.models import Segmentation_Network_full
from scripts.utils import save_mesh_with_color_clinical, save_mesh_with_color_acdc, dir_checker
from scripts.dataset import CustomDataset
from scripts import postprocessing

import torch
import torch.nn as nn


In [2]:
model_dir = './models/clinical_no_TL'
path_out = './predictions/Clinical'
target = 'best' # 'last'
model = torch.load(f'{model_dir}/{target}.pt').cpu()
model.eval()
print()

In [3]:
dir_in = './data_for_transfer_learning/obj/'
test_names = ['003', '009', '011', '012', '014', '016', '017', '021', '023', '026', '029', '030', '031', '032', '033', '034', '035', '037']

#name = random.choice(test_names) + '.obj'
name = '030.obj'

preprocessed_dir = 'TL_prepared_data/sdf'
preprocessed_dir += '/' + name[:-4]

mesh = trimesh.load(dir_in+name)
print('Infering on', name)

Infering on 030.obj


In [4]:
for name in test_names:
    name += '.obj'
    
    preprocessed_dir = 'TL_prepared_data/sdf'
    preprocessed_dir += '/' + name[:-4]

    mesh = trimesh.load(dir_in+name)


    if os.path.exists(preprocessed_dir):
        sdf = torch.load(f'{preprocessed_dir}/sdf.pt')
        colors = torch.load(f'{preprocessed_dir}/color.pt')
    else:
        datagen = CustomDataset(name = name,
                          device='cpu',
                          count=20,
                          rotate_angle=1,
                          dataset = 'clinical',
                          num_classes=4,
                          inference = True,
                          return_points = True,
                          data_dir = './data_for_transfer_learning/obj')
        
        sdf, colors, points = datagen.get_data()
        os.mkdir(preprocessed_dir)
        torch.save(sdf, f'{preprocessed_dir}/sdf.pt')
        torch.save(colors, f'{preprocessed_dir}/color.pt')
    
    predicted = model(sdf.float())
    
    colors = torch.argmax(colors, dim=1)
    predicted = torch.argmax(predicted, dim=1)
    
    dirname = dir_checker(path_out, name)
    save_mesh_with_color_clinical(mesh, colors, path=f'{dirname}/{name[:-4]}_original.obj')
    save_mesh_with_color_clinical(mesh, predicted, path=f'{dirname}/{name[:-4]}_predicted.obj')
    
    mesh = trimesh.load(f'{dirname}/{name[:-4]}_predicted.obj')
    
    postprocessing(mesh, f'{dirname}/{name[:-4]}_postprocessed.obj')

Folder for predictions: ./predictions/Clinical/003
Folder for predictions: ./predictions/Clinical/009
Folder for predictions: ./predictions/Clinical/011
Folder for predictions: ./predictions/Clinical/012
Folder for predictions: ./predictions/Clinical/014
Folder for predictions: ./predictions/Clinical/016
Folder for predictions: ./predictions/Clinical/017
Folder for predictions: ./predictions/Clinical/021
Folder for predictions: ./predictions/Clinical/023
Folder for predictions: ./predictions/Clinical/026
Folder for predictions: ./predictions/Clinical/029
Folder for predictions: ./predictions/Clinical/030
Folder for predictions: ./predictions/Clinical/031
Folder for predictions: ./predictions/Clinical/032
Folder for predictions: ./predictions/Clinical/033
Folder for predictions: ./predictions/Clinical/034
Folder for predictions: ./predictions/Clinical/035
Folder for predictions: ./predictions/Clinical/037


# Inference on 038-057

In [5]:
import numpy as np
import pyvista as pv


import os
import trimesh
import random

from tqdm import tqdm

from scripts.models import Segmentation_Network_full
from scripts.utils import save_mesh_with_color_clinical, save_mesh_with_color_acdc, dir_checker
from scripts.dataset import CustomDataset
from scripts import postprocessing

import torch
import torch.nn as nn



In [6]:
folder = './data_for_inference/'

In [7]:
dir_in = folder + 'vtu/'
dir_out = folder + 'obj/'

In [8]:
for i in os.listdir(dir_in):
    mesh = pv.read(dir_in + i)    
    surface = mesh.extract_surface()
    name = i[:-4]

    path = dir_out + name + '.obj'
    pv.save_meshio(filename= path, mesh = surface, file_format='obj')

In [9]:
model_dir = './models/clinical_no_TL'
path_out = './predictions/inference'
target = 'best' # 'last'
model = torch.load(f'{model_dir}/{target}.pt').cpu()
model.eval()
print()

In [10]:
dir_in = folder + 'obj/'
names = os.listdir(dir_in)

In [11]:
sorted(names)

['038.obj',
 '039.obj',
 '040.obj',
 '041.obj',
 '042.obj',
 '043.obj',
 '044.obj',
 '045.obj',
 '046.obj',
 '047.obj',
 '048.obj',
 '049.obj',
 '050.obj',
 '051.obj',
 '052.obj',
 '053.obj',
 '054.obj',
 '055.obj',
 '056.obj',
 '057.obj']

In [12]:
for name in tqdm(names):   
    preprocessed_dir = 'TL_prepared_data/sdf'
    preprocessed_dir += '/' + name[:-4]

    mesh = trimesh.load(dir_in+name)


    if os.path.exists(preprocessed_dir):
        sdf = torch.load(f'{preprocessed_dir}/sdf.pt')
        colors = torch.load(f'{preprocessed_dir}/color.pt')
    else:
        datagen = CustomDataset(name = name,
                          device='cpu',
                          count=20,
                          rotate_angle=1,
                          dataset = 'clinical',
                          num_classes=4,
                          inference = True,
                          return_points = True,
                          data_dir = dir_in)
        
        sdf, colors, points = datagen.get_data()
        os.mkdir(preprocessed_dir)
        torch.save(sdf, f'{preprocessed_dir}/sdf.pt')
        torch.save(colors, f'{preprocessed_dir}/color.pt')
    
    predicted = model(sdf.float())
    
    colors = torch.argmax(colors, dim=1)
    predicted = torch.argmax(predicted, dim=1)
    
    dirname = dir_checker(path_out, name)
    save_mesh_with_color_clinical(mesh, colors, path=f'{dirname}/{name[:-4]}_original.obj')
    save_mesh_with_color_clinical(mesh, predicted, path=f'{dirname}/{name[:-4]}_predicted.obj')
    
    mesh = trimesh.load(f'{dirname}/{name[:-4]}_predicted.obj')
    
    postprocessing(mesh, f'{dirname}/{name[:-4]}_postprocessed.obj')

  0%|                                                    | 0/20 [00:00<?, ?it/s]

Folder for predictions: ./predictions/inference/040


  5%|██▏                                         | 1/20 [00:00<00:15,  1.23it/s]

Folder for predictions: ./predictions/inference/042


 10%|████▍                                       | 2/20 [00:01<00:16,  1.12it/s]

...

 90%|██████████████████████████████████████▋    | 18/20 [00:18<00:01,  1.09it/s]

Folder for predictions: ./predictions/inference/049


 95%|████████████████████████████████████████▊  | 19/20 [00:19<00:01,  1.10s/it]

Folder for predictions: ./predictions/inference/046


100%|███████████████████████████████████████████| 20/20 [00:20<00:00,  1.04s/it]


# Clinical

In [2]:
model_dir = './models/clinical_no_TL'
path_out = './predictions/Clinical'
target = 'best' # 'last'
model = torch.load(f'{model_dir}/{target}.pt').cpu()
model.eval()
print()

In [3]:
dir_in = './data_for_transfer_learning/obj/'
test_names = ['003', '009', '011', '012', '014', '016', '017', '021', '023', '026', '029', '030', '031', '032', '033', '034', '035', '037']

name = random.choice(test_names) + '.obj'

preprocessed_dir = 'TL_prepared_data/sdf'
preprocessed_dir += '/' + name[:-4]

mesh = trimesh.load(dir_in+name)
print('Infering on', name)

Infering on 026.obj


In [4]:
if os.path.exists(preprocessed_dir):
    sdf = torch.load(f'{preprocessed_dir}/sdf.pt')
    colors = torch.load(f'{preprocessed_dir}/color.pt')
else:
    datagen = CustomDataset(name = name,
                      device='cpu',
                      count=20,
                      rotate_angle=1,
                      dataset = 'clinical',
                      num_classes=4,
                      inference = True,
                      return_points = True,
                      data_dir = './data_for_transfer_learning/obj')
    
    sdf, colors, points = datagen.get_data()
    os.mkdir(preprocessed_dir)
    torch.save(sdf, f'{preprocessed_dir}/sdf.pt')
    torch.save(colors, f'{preprocessed_dir}/color.pt')
    

In [9]:
predicted = model(sdf.float())

In [10]:
colors = torch.argmax(colors, dim=1)
predicted = torch.argmax(predicted, dim=1)

In [11]:
dirname = dir_checker(path_out, name)
save_mesh_with_color_clinical(mesh, colors, path=f'{dirname}/{name[:-4]}_original.obj')
save_mesh_with_color_clinical(mesh, predicted, path=f'{dirname}/{name[:-4]}_predicted.obj')

Folder for predictions: ./predictions/Clinical/017


In [20]:
class Pipeline:
    def __init__(self, model):
        """
        Pipeline to segment and postprocess 3d models
        Usage:
        pipeline = Pipeline(model)
        mesh = pipeline.run(path_in, file_name, preprocessed_dir)
        pipeline.save(path_out)
        """
        self.model = model

    def load_mesh(self, path_in, file_name, preprocessed_dir):
        if preprocessed_dir and os.path.exists(preprocessed_dir):
            sdf = torch.load(f'{preprocessed_dir}/sdf.pt')
            colors = torch.load(f'{preprocessed_dir}/color.pt')
        else:
            datagen = CustomDataset(name = file_name,
                              device='cpu',
                              count=20,
                              rotate_angle=1,
                              dataset = 'clinical',
                              num_classes=4,
                              inference = True,
                              return_points = True,
                              data_dir = path_in)
            
            sdf, colors, points = datagen.get_data()

            if preprocessed_dir:
                if os.path.exists(preprocessed_dir):
                    pass
                else:
                    os.mkdir(preprocessed_dir)
                torch.save(sdf, f'{preprocessed_dir}/sdf.pt')
                torch.save(colors, f'{preprocessed_dir}/color.pt')
        return sdf, colors 
    
    def run(self, path_in, file_name, preprocessed_dir=None):
        """
        Main function of the Pipeline, loads 3d obj in memory and processes it
        path_in - path to a folder with .obj file that has to be processed. Example: ./foo
        file_name - name of an .obj to that has to be processed. Example: bar.obj
        preprocessed_dir - path to directory with sdf files, if there're none, then they will be saved there
        """
        self.mesh = trimesh.load(dir_in+name)
        sdf, colors  = self.load_mesh(path_in, file_name, preprocessed_dir)
        
        predicted = self.model(sdf.float())

        self.colors = torch.argmax(colors, dim=1)
        self.predicted_colors = torch.argmax(predicted, dim=1)


    def save(self, path_out, file_name):
        """
        Saves the last processed mesh to a file
        """
        dirname = dir_checker(path_out, file_name)
        save_mesh_with_color_clinical(self.mesh, self.colors, path=f'{dirname}/{file_name[:-4]}_original.obj')
        save_mesh_with_color_clinical(self.mesh, self.predicted_colors, path=f'{dirname}/{file_name[:-4]}_predicted.obj')

In [21]:
pipeline = Pipeline(model)

In [25]:
for name in test_names:
    name += '.obj'
    mesh = pipeline.run(path_in=dir_in, file_name=name, preprocessed_dir=preprocessed_dir)
    pipeline.save(path_out, name)  

Folder for predictions: ./predictions/Clinical/003


IndexError: index 6367 is out of bounds for axis 0 with size 6367

In [23]:
pipeline.save(path_out, name)  

There's already a dirrectory with this name!
Folder for predictions: ./predictions/Clinical/009 (1)
